In [4]:
import pandas as pd
import numpy as np

In [5]:
# Avaliacoes
cols_ratings = ['userID', 'movieID', 'rating']
ratings = pd.read_csv('movies/user_ratedmovies.dat', sep='\t', error_bad_lines=False, usecols=cols_ratings)

# Filmes
cols_movies = ['movie_id', 'movie_title']
movies = pd.read_csv('movies/movies.dat', sep='\t', error_bad_lines=False, encoding='latin-1')
movies = movies.iloc[:,:2]
movies.columns = ['movieID', 'title']

In [6]:
ratings.head()

,userID,movieID,rating
0,75,3,1.0
1,75,32,4.5
2,75,110,4.0
3,75,160,2.0
4,75,163,4.0


In [83]:
# ratings.at[ratings['userID'] == 75, ratings['movieID'] == 3]
((ratings['userID'] == 75) & (ratings['movieID'] == 4)).any()

False

In [157]:
movies.head()

,movieID,title
0,1,Toy story
1,2,Jumanji
2,3,Grumpy Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II


In [7]:
combine_movies_ratings = pd.merge(ratings, movies, on='movieID')[['userID', 'title', 'movieID','rating']]
combine_movies_ratings.head()
#group = pd.DataFrame(ratings.groupby('userID')['movieID'].count())
# group.head()

,userID,title,movieID,rating
0,75,Grumpy Old Men,3,1.0
1,783,Grumpy Old Men,3,2.0
2,788,Grumpy Old Men,3,3.5
3,1160,Grumpy Old Men,3,4.0
4,1174,Grumpy Old Men,3,2.5


In [85]:
from scipy.sparse import csr_matrix

In [15]:
ratings_matriz = combine_movies_ratings.pivot_table(index='userID', columns='title', values='rating').fillna(0)
# movie_index = ratings_matriz.columns

In [16]:
ratings_matriz.head()

title,'Breaker' Morant,'Crocodile' Dundee II,'I Know Where I'm Going!','Northwest Passage' (Book I -- Rogers' Rangers),'Round Midnight,'Til There Was You,"'night, Mother",*batteries not included,...All the Marbles,...And Justice for All.,...,Äideistä parhain,Änglagård,Æon Flux,Éloge de l'amour,Éxtasis,Être et avoir,Ôdishon,Ônibus 174,Ôritsu uchûgun Oneamisu no tsubasa,Üç maymun
userID,,,,,,,,,,,,,,,,,,,,,
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
array = ratings_matriz

len(array.index)

2113

In [18]:
# Quantidade de usuário, filmes
ratings_matriz.values.shape

(2113, 9330)

In [276]:
ratings_space = csr_matrix(ratings_matriz.values)
ratings_space

<2114x9330 sparse matrix of type '<class 'numpy.float64'>'
	with 849040 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
array.index[600]

19909

In [77]:
knn.fit(array.values[0:500], array.index[0:500].astype('int'))
pred = knn.predict(array.values[1002].reshape(1, -1))
pred

array([1496])

In [78]:
t_sorted = array.loc[pred]
keys = t_sorted.columns
values = t_sorted.values.tolist()
# values
# d = dict(enumerate(values.flatten(), ))
# d
result = dict(zip(keys, values[0]))
# result
#type(result['10,000 BC'])
sorted_keys = sorted(result, key=result.__getitem__, reverse=True)
sorted_keys[0:11]
#t.values
#(combine_movies_ratings.sort_values('rating').groupby('userID').head(5)).title

['Aladdin',
 'American History X',
 'Batman Begins',
 'Casino Royale',
 'Collateral',
 'Crash',
 'Eternal Sunshine of the Spotless Mind',
 'Garden State',
 'Grosse Pointe Blank',
 'Hotaru no haka',
 "Le fabuleux destin d'Amélie Poulain"]

In [70]:
result['21 Grams']

5.0

In [12]:
x = (combine_movies_ratings.groupby('userID'))
x.head()

,userID,title,movieID,rating
0,75,Grumpy Old Men,3,1.0
1,783,Grumpy Old Men,3,2.0
2,788,Grumpy Old Men,3,3.5
3,1160,Grumpy Old Men,3,4.0
4,1174,Grumpy Old Men,3,2.5
5,1224,Grumpy Old Men,3,4.0
6,1632,Grumpy Old Men,3,3.0
7,1860,Grumpy Old Men,3,3.0
8,1915,Grumpy Old Men,3,3.0
9,2053,Grumpy Old Men,3,2.0
